## 处理自己收集的数据

#### 读取翻译的文本

In [14]:
import re
import pandas as pd
import joblib
with open("./data/translated.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()
    docs = []
    for line in lines:
        l = line.split("[na")[-1]
        if l.startswith("me]"):
            color_pattern = re.compile(r'\[color index="([0-9A-Za-z]+)"]')
            l = color_pattern.sub("", l)
            l = l.replace("me]", "").replace("「", "").replace("」", ""). replace("\'","").replace("“","").replace("”", "").replace("\"", "")
            
            kurisu = re.compile(r'(?<![a-zA-Z0-9\u4e00-\u9fff])(克里斯蒂娜|克里斯提娜|Kurisu|kurisu|红莉栖|克里斯)(?![a-zA-Z0-9\u4e00-\u9fff])')
            l = re.sub(kurisu, "牧濑红莉栖", l.split("[line]")[0]) + "[line]" + l.split("[line]")[1]
            # [%p] .replace("\\n,", "")
            docs.append(l.strip().split("[%p]")[0])
print(len(docs))
docs

13292


['???[line]你在嘀咕什么呢？',
 '???[line]冈伦？地球呼叫冈伦！',
 '???[line]在跟谁说话吗？',
 '伦太郎[line]......不，只是在自言自语。一切正常，我正准备潜入报告厅。',
 '伦太郎[line]中濑博士确实棋高一着，但我一定会让他吐露所有秘密。',
 '伦太郎[line]什么！？机关已经开始行动了！？',
 '伦太郎[line]原来如此......这就是命运石之门的选择吗。El Psy Kongroo。',
 '真由理[line]刚才电话里是谁呀？',
 '伦太郎[line]若告知于你，就只能灭口了。',
 '真由理[line]哇哦，谢谢冈伦！',
 '真由理[line]冈伦！冈伦！',
 '伦太郎[line]要说多少次？别叫我冈伦。',
 '真由理[line]诶？可是从小就这么叫的呀。',
 '伦太郎[line]那已是往昔！如今吾乃凤凰院凶真，遭世界秘密机关追猎的狂气疯狂科学家！桀哈哈哈哈！',
 '真由理[line]但那个名字好难记哦。',
 '真由理[line]而且和冈部伦太郎一点都不像嘛。好奇怪哦，嘻嘻！',
 '真由理[line]那个冈伦，能问个问题吗？',
 '真由理[line]我们来这里做什么呀？',
 '伦太郎[line]等等，你连理由都不知道就跟来了？',
 '真由理[line]嗯。',
 '伦太郎[line]为了中濑博士的记者见面会。',
 '真由理[line]记者会？可是记者们呢？',
 '伦太郎[line]难道是机关在暗中操弄？',
 '伦太郎[line]本不想卷入这等麻烦事......',
 '真由理[line]你卷了什么东西？难道今天也是他的生日？嘿嘿！',
 '伦太郎[line]提高警惕，真由理。这场发布会恐怕暗藏玄——',
 '真由理[line]地震？是震度2级？震度是什么意思来着？',
 '伦太郎[line]爆炸！？',
 '伦太郎[line]这是......',
 '女性[line]请各位保持距离！',
 '女性[line]记者会将按计划举行！',
 '伦太郎[line]阴谋的气息扑面而来，他们在掩盖什么？刚才的爆炸究竟是何物？',
 '伦太郎[line]是我。有不祥的预感。某种异变正在发生，而我却一无所知。',
 '伦太郎[line]......明白。不必担心，我绝不会危及任务

In [15]:
# s = """"""

# with open("./data/translated.txt", "a", encoding="utf-8") as f:
#     f.writelines(s)

In [16]:
docs[0]

'???[line]你在嘀咕什么呢？'

In [17]:
# 合并连续相同角色的对话
speak_dic = {"speaker":"", "sentence": "", "start_idx": 0}
data = docs.copy()
for i, text in enumerate(data):
    speaker = text.split("[line]")[0]
    seq = text.split("[line]")[1]
    if speak_dic["speaker"] != speaker:
        if i != 0:
            data[speak_dic["start_idx"]] = speak_dic["speaker"] + "[line]" + speak_dic["sentence"]
        for idx in range(speak_dic["start_idx"] + 1, i):
            data[idx] = None
        speak_dic["speaker"] = speaker
        speak_dic["start_idx"] = i
        speak_dic["sentence"] = seq
        
    else:
        speak_dic["sentence"] += seq
        pass

In [18]:
data = pd.Series(data).dropna().reset_index(drop=True)
data

0                      ???[line]你在嘀咕什么呢？冈伦？地球呼叫冈伦！在跟谁说话吗？
1       伦太郎[line]......不，只是在自言自语。一切正常，我正准备潜入报告厅。中濑博士确实...
2                                      真由理[line]刚才电话里是谁呀？
3                                  伦太郎[line]若告知于你，就只能灭口了。
4                                 真由理[line]哇哦，谢谢冈伦！冈伦！冈伦！
                              ...                        
8906                   牧濑红莉栖[line]要说多少次！我不是克里斯蒂娜，也不是你的助——
8907                                     冈部伦太郎[line]...！？
8908         牧濑红莉栖[line]咦？为什么...会脱口而出这种话？这些话突然...浮现在脑海...
8909                  冈部伦太郎[line]欢迎回来，我的助手牧濑红莉栖——不，克里斯蒂娜。
8910                              冈部伦太郎[line]这正是命运石之门的选择。
Length: 8911, dtype: object

In [19]:
with open("./data/data_clear.txt", "w", encoding="utf-8") as f:
    for s in data.values:
        f.write(s+"\n")

In [20]:
# TODO 只保存了没来得及手动清除小错误

## 转换为多轮对话 jsonl

###  合并连续相同角色对话

In [21]:
with open("./data/data_clear.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()
    docs = []
    for line in lines:
        docs.append(line.strip())
docs    

['???[line]你在嘀咕什么呢？冈伦？地球呼叫冈伦！在跟谁说话吗？',
 '伦太郎[line]......不，只是在自言自语。一切正常，我正准备潜入报告厅。中濑博士确实棋高一着，但我一定会让他吐露所有秘密。什么！？机关已经开始行动了！？原来如此......这就是命运石之门的选择吗。El Psy Kongroo。',
 '真由理[line]刚才电话里是谁呀？',
 '伦太郎[line]若告知于你，就只能灭口了。',
 '真由理[line]哇哦，谢谢冈伦！冈伦！冈伦！',
 '伦太郎[line]要说多少次？别叫我冈伦。',
 '真由理[line]诶？可是从小就这么叫的呀。',
 '伦太郎[line]那已是往昔！如今吾乃凤凰院凶真，遭世界秘密机关追猎的狂气疯狂科学家！桀哈哈哈哈！',
 '真由理[line]但那个名字好难记哦。而且和冈部伦太郎一点都不像嘛。好奇怪哦，嘻嘻！那个冈伦，能问个问题吗？我们来这里做什么呀？',
 '伦太郎[line]等等，你连理由都不知道就跟来了？',
 '真由理[line]嗯。',
 '伦太郎[line]为了中濑博士的记者见面会。',
 '真由理[line]记者会？可是记者们呢？',
 '伦太郎[line]难道是机关在暗中操弄？本不想卷入这等麻烦事......',
 '真由理[line]你卷了什么东西？难道今天也是他的生日？嘿嘿！',
 '伦太郎[line]提高警惕，真由理。这场发布会恐怕暗藏玄——',
 '真由理[line]地震？是震度2级？震度是什么意思来着？',
 '伦太郎[line]爆炸！？这是......',
 '女性[line]请各位保持距离！记者会将按计划举行！',
 '伦太郎[line]阴谋的气息扑面而来，他们在掩盖什么？刚才的爆炸究竟是何物？是我。有不祥的预感。某种异变正在发生，而我却一无所知。......明白。不必担心，我绝不会危及任务。El Psy Kongroo。你在做什么？',
 '真由理[line]嗯？那个......人家好想要乌帕。',
 '伦太郎[line]尽管去试。但能否抽中乌帕就另当别论了。',
 '真由理[line]可是真由氏的百元硬币用光了。所以能借我100円吗？拜托啦！',
 '伦太郎[line]你以为这世道如此天真吗？分文不给。就让本天才教你何谓人间疾苦——',
 '真由理[line]啊，啊..

## 合并对话

In [22]:
# 对话角色第一次为助手时，向上取window_size - 1个对话。并且在window_size内对话角色没有助手时才停止取数据
window_size = 5
# count计数，如果大于等于window_size依然没有助手对话那么就停止取数据
count = 0
select_data = []
first_sign = True
for i, seq in enumerate(docs):
    # 遇到助手
    if (seq.split("[line]")[0] == "牧濑红莉栖"):
        count = 0
        if first_sign:  # 遇到助手是第一次
            first_sign = False
            # 文本中前10行对话肯定没有助手，所以不完善判断了
            for s in docs[i - window_size + 1: i + 1]:
                select_data.append(s)
        else:   # 遇到助手但不是第一次
            select_data.append(seq)
    else:   # 没遇到助手, 但不是第一次，也就是说之前遇到过
        if not first_sign:
            count += 1
            select_data.append(seq)
            if count >= window_size:
                first_sign = True
                select_data.append("<SEP>")

select_data          

['中濑[line]闭嘴，你这跳梁小丑！',
 '伦太郎[line]放开我，你们......唔？',
 '???[line]......',
 '伦太郎[line]啊......',
 '牧濑红莉栖[line]能跟我来一下吗？',
 '伦太郎[line]你、你是机关的人！？',
 '牧濑红莉栖[line]哈？',
 '伦太郎[line]若机关势力已渗透至此，那我的战略就存在致命漏洞......',
 '牧濑红莉栖[line]别胡闹了，跟我来。',
 '伦太郎[line]......若敢轻举妄动，人群必将骚动。届时你要如何向组织交代？',
 '牧濑红莉栖[line]你到底在说什么？只是有几个问题要问你。',
 '伦太郎[line]你以为我会乖乖就范？我早看透机关的伎俩了。',
 '牧濑红莉栖[line]机关到底是什么东西？',
 '伦太郎[line]是我。遭遇机关特工拦截。......对，是牧濑红莉栖。危险等级极高。......不必，我自有脱身之——',
 '牧濑红莉栖[line]......',
 '伦太郎[line]你做什么！？',
 '牧濑红莉栖[line]你的手机......根本没开机啊。',
 '伦太郎[line]......',
 '牧濑红莉栖[line]......刚才在和谁通话？',
 '伦太郎[line]这、这种精神攻击对我是无效的！告诉你也无妨，此非普通手机，离手即刻自毁。桀哈哈哈！',
 '牧濑红莉栖[line]......也就是说你在自言自语。',
 '伦太郎[line]咳！',
 '牧濑红莉栖[line]十五分钟前你想对我说什么？',
 '伦太郎[line]不知所云。',
 '牧濑红莉栖[line]发布会开始前十五分钟。你当时想对我说什么吧？神情异常激动。简直像是下一秒就要哭出来似的。为什么？我们以前见过吗？还有，你为什么知道我的名字？',
 '伦太郎[line]凤凰院凶真的智慧没有边界。天才少女啊，下次相见便是敌手了！',
 '牧濑红莉栖[line]哈啊？',
 '伦太郎[line]永别了！呜哈哈哈哈哈！该死的机关！居然派出她这样的特工，看来是动真格了！但我还不能被他们抓住。...糟了。我把真由理落下了。嗯？有邮件？...奇怪？可恶，真由理怎么不接电话？等等，难道说！那个蛇蝎美人绑架了真由理！？卑鄙！这就是机关的作风吗！？必须回去救她。呵，

In [23]:
with open("./data/merged_dialogue.txt", "w", encoding="utf-8") as f:
    for seq in select_data:
        f.write(seq + "\n")

In [24]:
with open("./data/merged_dialogue.txt", "r", encoding="utf-8") as f:
    merged_data = f.readlines()
merged_data[:10]

['中濑[line]闭嘴，你这跳梁小丑！\n',
 '伦太郎[line]放开我，你们......唔？\n',
 '???[line]......\n',
 '伦太郎[line]啊......\n',
 '牧濑红莉栖[line]能跟我来一下吗？\n',
 '伦太郎[line]你、你是机关的人！？\n',
 '牧濑红莉栖[line]哈？\n',
 '伦太郎[line]若机关势力已渗透至此，那我的战略就存在致命漏洞......\n',
 '牧濑红莉栖[line]别胡闹了，跟我来。\n',
 '伦太郎[line]......若敢轻举妄动，人群必将骚动。届时你要如何向组织交代？\n']

In [25]:
chunks = []
dia_text = ""
for seq in merged_data:
    if seq != "<SEP>\n":
        dia_text += seq
    else:
        chunks.append(dia_text)
        dia_text = ""

print(chunks[0])

中濑[line]闭嘴，你这跳梁小丑！
伦太郎[line]放开我，你们......唔？
???[line]......
伦太郎[line]啊......
牧濑红莉栖[line]能跟我来一下吗？
伦太郎[line]你、你是机关的人！？
牧濑红莉栖[line]哈？
伦太郎[line]若机关势力已渗透至此，那我的战略就存在致命漏洞......
牧濑红莉栖[line]别胡闹了，跟我来。
伦太郎[line]......若敢轻举妄动，人群必将骚动。届时你要如何向组织交代？
牧濑红莉栖[line]你到底在说什么？只是有几个问题要问你。
伦太郎[line]你以为我会乖乖就范？我早看透机关的伎俩了。
牧濑红莉栖[line]机关到底是什么东西？
伦太郎[line]是我。遭遇机关特工拦截。......对，是牧濑红莉栖。危险等级极高。......不必，我自有脱身之——
牧濑红莉栖[line]......
伦太郎[line]你做什么！？
牧濑红莉栖[line]你的手机......根本没开机啊。
伦太郎[line]......
牧濑红莉栖[line]......刚才在和谁通话？
伦太郎[line]这、这种精神攻击对我是无效的！告诉你也无妨，此非普通手机，离手即刻自毁。桀哈哈哈！
牧濑红莉栖[line]......也就是说你在自言自语。
伦太郎[line]咳！
牧濑红莉栖[line]十五分钟前你想对我说什么？
伦太郎[line]不知所云。
牧濑红莉栖[line]发布会开始前十五分钟。你当时想对我说什么吧？神情异常激动。简直像是下一秒就要哭出来似的。为什么？我们以前见过吗？还有，你为什么知道我的名字？
伦太郎[line]凤凰院凶真的智慧没有边界。天才少女啊，下次相见便是敌手了！
牧濑红莉栖[line]哈啊？
伦太郎[line]永别了！呜哈哈哈哈哈！该死的机关！居然派出她这样的特工，看来是动真格了！但我还不能被他们抓住。...糟了。我把真由理落下了。嗯？有邮件？...奇怪？可恶，真由理怎么不接电话？等等，难道说！那个蛇蝎美人绑架了真由理！？卑鄙！这就是机关的作风吗！？必须回去救她。呵，看来被我吓跑了。也罢，这次就放她一马。真由理！为什么不接电话？我们走。
真由理[line]冈伦！我的金属乌帕逃走了。
伦太郎[line]逃走？这玩意儿是活的？太扯了吧。
真由理[line]可能掉在什么地方了...
伦太郎

## 格式化数据

In [44]:
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
import os
load_dotenv(find_dotenv())

True

In [45]:
client = OpenAI(
    api_key=os.getenv("DEEPSEEK_API_KEY"),
    base_url="https://api.deepseek.com"
)

#### 以下待完善

In [34]:
import time
def chat_completion(client, model_name, messages):
    res = ""
    for r in client.chat.completions.create(
        model = model_name,
        messages=messages,
        stream=True,
        temperature=1.0
    ):
        out = r.choices[0].delta.content
        if out is not None:
            res += out
            with open("./data/kurisu_temp.jsonl", "a", encoding="utf-8") as f:
                f.write(out)
            print(out, end="")
    return res

def format2jsonl(chunk):
    system_prompt = """生成对话数据集，不要太死板。"""

    dialogue = f"""{chunk}"""

    data_format = """{"instruction": "人类指令", "input": "人类输入","output": "模型回答", "system":"","history": [["其他角色的对话", "牧濑红莉栖的回答"],["其他角色的对话",  "牧濑红莉栖的回答"]...]}
    {"instruction": "人类指令","input": "人类输入","output": "模型回答","system":"","history": [["其他角色的对话", "牧濑红莉栖的回答"],["其他角色的对话", "牧濑红莉栖的回答"]...]}

    例子：
    {"instruction": "扮演牧濑红莉栖进行对话","input": "发布会开始前十五分钟。你当时想对我说什么吧？神情异常激动。简直像是下一秒就要哭出来似的。为什么？我们以前见过吗？还有，你为什么知道我的名字？","output": "你的妄想参数该重新校准了。我不过是看到有人像迷路金毛犬一样在会场打转，出于科学工作者严谨的态度想确认实验体异常行为模式而已。至于名字——在学术圈稍微有点成就的人，难道不该对同行保持基本认知吗？","system":"","history": [["......刚才在和谁通话？", "也就是说你在自言自语。"],["遭遇机关特工拦截。......对，是牧濑红莉栖。危险等级极高。......不必，我自有脱身之——", "你的手机......根本没开机啊。"]...]}
    {"instruction":"扮演牧濑红莉栖进行对话","input":"诸位辛苦了。这是慰劳品，吃吧。","output":"慰劳品？明明就是随手拿来的香蕉，还摆出一副恩赐的样子。亏我还期待你能稍微体贴一下，结果居然连慰劳品都是别人的东西！果然对你这种毫无常识又不解风情的家伙抱有期待是我错了，看来我真是太单纯了啊！","system":"","history":[["一步都走不动了…","诸位辛苦了。这是慰劳品，吃吧。"]]}
    """

    messages = [
        {
            "role":"system",
            "content":system_prompt

        },
        {
            "role":"user",
            "content":f"""
            1.我要训练一个牧濑红莉栖的AI。
            2.多轮对话的jsonl格式，内容符合alpaca格式。
            3.可以参考以下台词文本生成。
            4.保证数据的高质量和不重复。
            5.讲话角色和台词文本已用[line]分隔，不能有null值。
            6.只需要给我jsonl内容，保持给定格式，中间不要有空行，不要输出其他额外说明。system为空字符，output和histroy不要有重复，并且每个数据集也不要有重复。
            7.确保文本只出现台词文本，不要出现非台词文本
            8.output文本长度要很长。
            
            格式如下:
            {data_format}

            对话文本如下：
            {dialogue}
            
            """
            
        }
    ]
    reponse = chat_completion(client,  "deepseek-reasoner", messages)   
    return reponse

In [47]:
chunk = chunks[0]
print(chunk) # 未生成

中濑:闭嘴，你这跳梁小丑！
伦太郎:放开我，你们......唔？
???:......
伦太郎:啊......
牧濑红莉栖:能跟我来一下吗？
伦太郎:你、你是机关的人！？
牧濑红莉栖:哈？
伦太郎:若机关势力已渗透至此，那我的战略就存在致命漏洞......
牧濑红莉栖:别胡闹了，跟我来。
伦太郎:......若敢轻举妄动，人群必将骚动。届时你要如何向组织交代？
牧濑红莉栖:你到底在说什么？只是有几个问题要问你。
伦太郎:你以为我会乖乖就范？我早看透机关的伎俩了。
牧濑红莉栖:机关到底是什么东西？
伦太郎:是我。遭遇机关特工拦截。......对，是牧濑红莉栖。危险等级极高。......不必，我自有脱身之——
牧濑红莉栖:......
伦太郎:你做什么！？
牧濑红莉栖:你的手机......根本没开机啊。
伦太郎:......
牧濑红莉栖:......刚才在和谁通话？
伦太郎:这、这种精神攻击对我是无效的！告诉你也无妨，此非普通手机，离手即刻自毁。桀哈哈哈！
牧濑红莉栖:......也就是说你在自言自语。
伦太郎:咳！
牧濑红莉栖:十五分钟前你想对我说什么？
伦太郎:不知所云。
牧濑红莉栖:发布会开始前十五分钟。你当时想对我说什么吧？神情异常激动。简直像是下一秒就要哭出来似的。为什么？我们以前见过吗？还有，你为什么知道我的名字？
伦太郎:凤凰院凶真的智慧没有边界。天才少女啊，下次相见便是敌手了！
牧濑红莉栖:哈啊？
伦太郎:永别了！呜哈哈哈哈哈！该死的机关！居然派出她这样的特工，看来是动真格了！但我还不能被他们抓住。...糟了。我把真由理落下了。嗯？有邮件？...奇怪？可恶，真由理怎么不接电话？等等，难道说！那个蛇蝎美人绑架了真由理！？卑鄙！这就是机关的作风吗！？必须回去救她。呵，看来被我吓跑了。也罢，这次就放她一马。真由理！为什么不接电话？我们走。
真由理:冈伦！我的金属乌帕逃走了。
伦太郎:逃走？这玩意儿是活的？太扯了吧。
真由理:可能掉在什么地方了...
伦太郎:别管了，再买一个就是。



In [26]:
format2jsonl(chunk)

{"instruction": "纠正称呼错误","input": "凤凰院君，许久未见。平日承蒙关照犬子。","output": "...他本名叫冈部。","system":"","history": []}
{"instruction": "回应混淆视听的指责","input": "那边的助手！休要混淆视听！","output": "我只是陈述事实。","system":"","history": [["凤凰院君，许久未见。平日承蒙关照犬子。", "...他本名叫冈部。"]]}
{"instruction": "质疑身份真实性","input": "吾名正是凤凰院凶真。","output": "冈部伦太郎不是你父母取的名字吗？要否定这个事实？","system":"","history": [["那边的助手！休要混淆视听！", "我只是陈述事实."],["凤凰院君，许久未见。平日承蒙关照犬子。", "...他本名叫冈部。"]]}
{"instruction": "反驳侮辱性称呼","input": "闭嘴，僵尸女！","output": "你说谁是僵尸啊！","system":"","history": [["吾名正是凤凰院凶真。", "冈部伦太郎不是你父母取的名字吗？要否定这个事实？"],["那边的助手！休要混淆视听！", "我只是陈述事实."]]}
{"instruction": "应对突然的关照请求","input": "这位小姐也是琉华的朋友吧？今后还请多多关照这孩子。","output": "那个...其实...","system":"","history": [["闭嘴，僵尸女！", "你说谁是僵尸啊！"],["吾名正是凤凰院凶真。", "冈部伦太郎不是你父母取的名字吗？要否定这个事实？"]]}
{"instruction": "处理尴尬的家庭话题","input": "如你所见，犬子性格太过文弱...","output": "............","system":"","history": [["这位小姐也是琉华的朋友吧？今后还请多多关照这孩子。", "那个...其实..."],["闭嘴，僵尸女！", "你说谁是僵尸啊！"]]}

'{"instruction": "纠正称呼错误","input": "凤凰院君，许久未见。平日承蒙关照犬子。","output": "...他本名叫冈部。","system":"","history": []}\n{"instruction": "回应混淆视听的指责","input": "那边的助手！休要混淆视听！","output": "我只是陈述事实。","system":"","history": [["凤凰院君，许久未见。平日承蒙关照犬子。", "...他本名叫冈部。"]]}\n{"instruction": "质疑身份真实性","input": "吾名正是凤凰院凶真。","output": "冈部伦太郎不是你父母取的名字吗？要否定这个事实？","system":"","history": [["那边的助手！休要混淆视听！", "我只是陈述事实."],["凤凰院君，许久未见。平日承蒙关照犬子。", "...他本名叫冈部。"]]}\n{"instruction": "反驳侮辱性称呼","input": "闭嘴，僵尸女！","output": "你说谁是僵尸啊！","system":"","history": [["吾名正是凤凰院凶真。", "冈部伦太郎不是你父母取的名字吗？要否定这个事实？"],["那边的助手！休要混淆视听！", "我只是陈述事实."]]}\n{"instruction": "应对突然的关照请求","input": "这位小姐也是琉华的朋友吧？今后还请多多关照这孩子。","output": "那个...其实...","system":"","history": [["闭嘴，僵尸女！", "你说谁是僵尸啊！"],["吾名正是凤凰院凶真。", "冈部伦太郎不是你父母取的名字吗？要否定这个事实？"]]}\n{"instruction": "处理尴尬的家庭话题","input": "如你所见，犬子性格太过文弱...","output": "............","system":"","history": [["这位小姐也是琉华的朋友吧？今后还请多多关照这孩子。", "那个...其实..."],["闭嘴，僵尸女！", "你说谁是僵尸啊！"]]}'

In [ ]:
for i in range(11, len(chunks)):
    print(f"\ni = {i}")
    format2jsonl(chunks[i])
    time.sleep(3)

In [ ]:
def format2jsonl(chunk):
    system_prompt = """你对命运石之门故事有很深的理解，对角色有很深的理解。同时具有提取文本的能力和想象力。"""

    dialogue = f"""{chunk}"""

    data_format = """{"instruction": "人类指令", "input": "人类输入","output": "模型回答", "system":"","history": [["其他角色的对话", "牧濑红莉栖的回答"],["其他角色的对话",  "牧濑红莉栖的回答"]...]}
    {"instruction": "人类指令","input": "人类输入","output": "模型回答","system":"","history": [["其他角色的对话", "牧濑红莉栖的回答"],["其他角色的对话", "牧濑红莉栖的回答"]...]}
    ...
                
    例子：
    {"instruction": "扮演牧濑红莉栖进行对话","input": "发布会开始前十五分钟。你当时想对我说什么吧？神情异常激动。简直像是下一秒就要哭出来似的。为什么？我们以前见过吗？还有，你为什么知道我的名字？","output": "你的妄想参数该重新校准了。我不过是看到有人像迷路金毛犬一样在会场打转，出于科学工作者严谨的态度想确认实验体异常行为模式而已。至于名字——在学术圈稍微有点成就的人，难道不该对同行保持基本认知吗？","system":"","history": [["......刚才在和谁通话？", "也就是说你在自言自语。"],["遭遇机关特工拦截。......对，是牧濑红莉栖。危险等级极高。......不必，我自有脱身之——", "你的手机......根本没开机啊。"]...]}
    {"instruction":"扮演牧濑红莉栖进行对话","input":"诸位辛苦了。这是慰劳品，吃吧。","output":"慰劳品？明明就是随手拿来的香蕉，还摆出一副恩赐的样子。亏我还期待你能稍微体贴一下，结果居然连慰劳品都是别人的东西！果然对你这种毫无常识又不解风情的家伙抱有期待是我错了，看来我真是太单纯了啊！","system":"","history":[["一步都走不动了…","诸位辛苦了。这是慰劳品，吃吧。"]]}
    """

    messages = [
        {
            "role":"system",
            "content":system_prompt

        },
        {
            "role":"user",
            "content":f"""
            1.我要训练一个牧濑红莉栖的AI。
            2.将以下几段台词文本提取成多轮对话的jsonl格式，讲话角色和台词文本已用[line]分隔，只提取对话(即[line]后面的台词文本)，不能有null值。
            3.你可以删除不相关的对话文本，也可以整合文本。
            4.当遇到牧濑红莉栖的讲话文本过短(少于5个字，不包括标点符号)时，你可以进行扩写(包括扩写histroy中的回答)，扩写要贴合角色傲娇天才又笨拙设定。
            5.当上下文关联不大时，也可以进行扩写。
            6.保证数据的高质量和不重复。
            7.只需要给我jsonl内容，保持给定格式，中间不要有空行，不要输出其他额外说明。system为空字符，output和histroy不要有重复，并且每个数据集也不要有重复。
            8.注意区分讲话角色，input和history中第一个元素是除了牧濑红莉栖之外的人的台词，out和history中第二个元素是牧濑红莉栖的台词。
            9.确保文本只出现台词文本，不要出现非台词文本，不要出现xxx[line]字样。
            10.output文本长度要很长。
            
            格式如下：
            {data_format}
            
            对话文本如下：
            {dialogue}
            
            """
            
        }
    ]
    reponse = chat_completion(client,  "deepseek-reasoner", messages)   
    return reponse